<a href="https://colab.research.google.com/github/OrangeLyx/507final_369/blob/main/code/train_part4_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DeBERTa training pipeline

In [ ]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install datasets
from datasets import load_dataset,Dataset
imdb = load_dataset("imdb")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
train_dataset=imdb["train"].shuffle(seed=123)
test_dataset=imdb["test"].shuffle(seed=123)

data_train= pd.DataFrame({"text": train_dataset["text"], "label":train_dataset["label"]})
data_test= pd.DataFrame({"text": test_dataset["text"], "label":test_dataset["label"]})

In [ ]:
def clean_text(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub(r'<.*?>','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub(r'[^a-zA-Z0-9\s]','',text)
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)

    stop_words = {'a', 'an', 'the'}
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

cleaned=lambda x:clean_text(x)

data_test['text']=pd.DataFrame(data_test.text.apply(cleaned))
data_train['text']=pd.DataFrame(data_train.text.apply(cleaned))


data_test.head(10)
data_train.head(10)

,text,label
0,i wont claim to be fan of ralph bakshi because...,1
1,i was not impressed about this film especially...,0
2,aka demons iiimade for italian tv although sho...,0
3,thief of baghdad is one of my ten alltime favo...,1
4,this is first 10 out of 10 that ive given any ...,1
5,there is uk edition to this show which is rath...,0
6,spoilersi saw original on tv sometime ago and ...,1
7,spoilers powerhouse movie that shows how men i...,1
8,this movie is intelligent that is more than mo...,1
9,despite unfortunately thinking itself to be in...,0


In [ ]:
data_all = pd.concat([data_train, data_test], axis=0).reset_index(drop=True)

data_train, data_test= train_test_split(data_all, test_size=0.25, random_state=42, stratify=data_all["label"])

print("Size of train dataset:", len(data_train))
print("Size of test dataset:", len(data_test))
print(data_train["label"].value_counts(normalize=True))
print(data_test["label"].value_counts(normalize=True))

dataset_train = Dataset.from_pandas(data_train)
dataset_test = Dataset.from_pandas(data_test)

训练集大小: 37500
验证集大小: 12500
label
0    0.5
1    0.5
Name: proportion, dtype: float64
label
0    0.5
1    0.5
Name: proportion, dtype: float64


In [ ]:
# Prepare the text inputs for the model
dataset_train = Dataset.from_pandas(data_train)
dataset_test = Dataset.from_pandas(data_test)

In [ ]:
import torch
from transformers import TrainingArguments, Trainer,AutoModelForSequenceClassification, AutoTokenizer
!pip install evaluate
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large", num_labels=2)
model.to(device)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, element

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_train = dataset_train.map(preprocess_function, batched=True)
tokenized_test = dataset_test.map(preprocess_function, batched=True)

tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])


Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_deberta",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_deberta",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
accuracy = evaluate.load("accuracy")
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    acc = accuracy_score(labels, predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="binary"
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.196500,0.177741,0.939120,0.932137,0.947200,0.939608
2,0.136800,0.266749,0.939360,0.933396,0.946240,0.939774
3,0.088400,0.275272,0.938800,0.941840,0.935360,0.938589


TrainOutput(global_step=7032, training_loss=0.14711531032135328, metrics={'train_runtime': 2933.1692, 'train_samples_per_second': 38.354, 'train_steps_per_second': 2.397, 'total_flos': 2.62107465408e+16, 'train_loss': 0.14711531032135328, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.26674890518188477,
 'eval_accuracy': 0.93936,
 'eval_precision': 0.9333964646464646,
 'eval_recall': 0.94624,
 'eval_f1': 0.9397743524551089,
 'eval_runtime': 89.6916,
 'eval_samples_per_second': 139.366,
 'eval_steps_per_second': 8.719,
 'epoch': 3.0}

In [ ]:
trainer.save_model("./model_deberta")
tokenizer.save_pretrained("./model_deberta")